In [ ]:
# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Carga y unión de los datos
Los datos de características químicas vienen en *wine-data.csv* y las etiquetas de cultivo en *wine-segments.csv*.

In [ ]:
# Ajustar la ruta si ejecutas desde otra carpeta
X_df = pd.read_csv('wine-data.csv')
y_df = pd.read_csv('wine-segments.csv')

print('Shape X:', X_df.shape)
print('Shape y:', y_df.shape)

df = X_df.copy()
df['Cultivar'] = y_df['Cultivar']
df.head()

## Análisis exploratorio de los datos (EDA)

In [ ]:
# Información general del dataset
df.info()

In [ ]:
# Estadísticos descriptivos de las variables numéricas
df.describe().T

In [ ]:
# Distribución de la variable objetivo (Cultivar)
cult_counts = df['Cultivar'].value_counts().sort_index()
print(cult_counts)

sns.countplot(data=df, x='Cultivar')
plt.title('Distribución de los cultivares')
plt.show()

In [ ]:
# Matriz de correlación de las variables químicas
plt.figure(figsize=(12, 10))
sns.heatmap(df.drop(columns=['Cultivar']).corr(), cmap='coolwarm', center=0)
plt.title('Matriz de correlación - variables químicas')
plt.show()

## Preparación de datos para modelado
Separamos variables predictoras (X) y la variable objetivo (y), y luego dividimos en entrenamiento y prueba.

In [ ]:
X = df.drop(columns=['Cultivar'])
y = df['Cultivar']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

## Entrenamiento con distintos modelos de Boosting
Probamos al menos dos modelos de boosting: **Gradient Boosting** y **AdaBoost**.
Se evalúan mediante validación cruzada sobre el conjunto de entrenamiento.

In [ ]:
models = {
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

results = []
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    results.append({
        'Modelo': name,
        'Accuracy_promedio': scores.mean(),
        'Accuracy_std': scores.std()
    })

pd.DataFrame(results)

## Ajuste de hiperparámetros con GridSearchCV (Gradient Boosting)
Se optimizan hiperparámetros clave como `n_estimators`, `learning_rate` y `max_depth` para el modelo de Gradient Boosting.

In [ ]:
gb_clf = GradientBoostingClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4]
}

grid_search = GridSearchCV(
    estimator=gb_clf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print('Mejores hiperparámetros:', grid_search.best_params_)
print('Mejor accuracy (CV):', grid_search.best_score_)

## Evaluación del mejor modelo en el conjunto de prueba

In [ ]:
best_model = grid_search.best_estimator_

y_pred_test = best_model.predict(X_test)

print('Accuracy en test:', accuracy_score(y_test, y_pred_test))
print('Reporte de clasificación:', classification_report(y_test, y_pred_test))

cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de confusión - Mejor Gradient Boosting')
plt.show()

## Comparación final y conclusiones
- Revise la tabla de resultados de validación cruzada para comparar Gradient Boosting vs AdaBoost.
- Compare esas métricas con el desempeño del mejor modelo optimizado en el conjunto de prueba.
- Analice si hay sobreajuste (CV vs test) y qué tan bien distingue entre los 3 cultivares.